In [122]:
import pandas as pd
import re

In [123]:
# DataFrame com as falas de Harry Potter
dialogues = pd.read_csv(r'C:\Users\DVILA\Downloads\hp-dashboard\hp-dashboard\data\Dialogue.csv', sep=',', encoding='latin1')

spells = pd.read_csv(r'C:\Users\DVILA\Downloads\hp-dashboard\hp-dashboard\data\Spells.csv')

chapter = pd.read_csv(r'C:\Users\DVILA\Downloads\hp-dashboard\hp-dashboard\data\Chapters.csv', sep=',', encoding='latin1')


In [124]:
chapter

,Chapter ID,Chapter Name,Movie ID,Movie Chapter
0,1,Doorstep Delivery,1,1
1,2,The Vanishing Glass,1,2
2,3,Letters from No One,1,3
3,4,Keeper of the Keys,1,4
4,5,Diagon Alley,1,5
...,...,...,...,...
229,230,Declare Yourself,8,22
230,231,Not Over,8,23
231,232,The Wand's Choice,8,24
232,233,The Wizard's Choice,8,25


In [125]:
 # DataFrame com as magias
df_falas = dialogues[['Dialogue', 'Character ID', 'Chapter ID']]
df_magias = spells['Incantation']
df_chapters = chapter[['Chapter ID', 'Movie ID']] 

correcoes = {"Avada": "Avada Kedavra"}

# Lista para armazenar as magias detectadas
spell_matches = []

# Loop para identificar magias nas falas
for index, row in df_falas.iterrows():
    fala = row['Dialogue']
    char_id = row['Character ID']
    chap_id = row['Chapter ID']
    
    # Obter o Movie ID correspondente ao Chapter ID
    movie_id = df_chapters.loc[df_chapters['Chapter ID'] == chap_id, 'Movie ID']
    movie_id = movie_id.iloc[0] if not movie_id.empty else None

    # Limpar a fala removendo caracteres especiais
    fala_limpa = re.sub(r'[^a-zA-Z0-9\s]', '', fala.lower())  

    # Aplicar correções personalizadas na fala limpa
    for palavra_errada, palavra_correta in correcoes.items():
        fala_limpa = fala_limpa.replace(palavra_errada, palavra_correta)
    
    # Verificar se a fala contém alguma magia corrigida
    for magia in df_magias:
        magia_lower = magia.lower()
        if magia_lower in fala_limpa:
            # Formatar a magia para ter letras maiúsculas no início de cada palavra
            magia_formatada = ' '.join([palavra.capitalize() for palavra in magia.split()])
            spell_matches.append({
                'Incantation': magia_formatada,
                'Character ID': char_id,
                'Chapter ID': chap_id,
                'Movie ID': movie_id
            })

# Criar um novo DataFrame com as falas e magias detectadas
df_novo = pd.DataFrame(spell_matches)

print(df_novo)

          Incantation  Character ID  Chapter ID  Movie ID
0              Finite            43           7         1
1       Oculus Reparo             3          10         1
2              Reparo             3          10         1
3           Alohomora             3          16         1
4           Alohomora             2          16         1
..                ...           ...         ...       ...
158     Avada Kedavra            99         223         8
159      Expelliarmus             1         223         8
160  Expecto Patronum             6         225         8
161         Confringo             1         231         8
162         Confringo             1         232         8

[163 rows x 4 columns]


In [126]:
output_path = r'C:\Users\DVILA\Downloads\hp-dashboard\hp-dashboard\data\SpellsDialogue.xlsx'

df_novo.to_excel(output_path, index=False, engine='openpyxl', sheet_name='SpellDialogue')